In [ ]:
# wrap the output in colab cells
from IPython.display import HTML, display
def set_css():
  display(HTML('''  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
# open and read the file from google drive
# https://github.com/gcunhase/AMICorpusXML/tree/master/data/ami-transcripts-speaker
file = open("/content/EN2002c.A.transcript.txt", "r")
FileContent = file.read().strip()

In [ ]:
# display file content
FileContent

"Did we get seven out of ten for the other one? Okay. Yep. Not bad. Um I think there is. Just because um the the topic box and all this is um is based on um, oh what's it called, nom no, NITE text area or something. In text area, is that what it is? Um it has some sort of highlighting stuff there. So you should probably look at the Um I'm not sure if it's in-built though, to be honest. What what what wh wh You don't. Okay, I'll do it. Yeah, I w I was thinking about uh I was going that direction anyway. Yeah, that Okay, yeah. So Okay, yeah, I see what you mean. I think. I think. So um I uh have you already calculated all this data or are you supposed to do that? Oh right. Alright. Uh what should it come back as? Numbers. Oh, yeah. Um whatever you give me I can do it, I think. I mean um Um Um so what will whenever we open a window for one speaker um Yeah, yeah. Sh pretty much, I would have thought. Yep. Yep. Definitely. Oh is it? Um the one present. Although we could do th mm we could do

In [ ]:
# total characters in the file
len(FileContent)

15826

In [ ]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# max tokens including the special tokens
tokenizer.model_max_length

1024

In [ ]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [ ]:
# number of special tokens
tokenizer.num_special_tokens_to_add()

2

In [ ]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

84

In [ ]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

4

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1004, 1016, 1002, 934]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[1006, 1018, 1004, 936]

In [ ]:
sum([len(tokenizer(c).input_ids) for c in chunks])

3964

In [ ]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (3958 > 1024). Running this sequence through the model will result in indexing errors


3958

In [ ]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

3956

In [ ]:
len(tokenizer.tokenize(FileContent))

3956

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

Right-click on a topic window on one of the topics, you have the option of pop popping open a window which gives you a list of all the meetings containing that topic. So you could you could zap through these topics uh meetings and it would come up with how much they spoke and you can pick the one that s looks the most interesting.
Right, well I've been doing some random pop-up windows for the speaker characterisation, which is good. The windows are all there now after lots of fiddling. I'm working on when you click on a topic to see a list of the of the topics of of the meetings that that topic was mentioned in. And I have uh a question about the we said we wanted a start screen. What kind of start screen do we want? Some animation dancing on.
But it would be No, but the general inverted file. That would just be a nice extra. But then yeah, as I said, how do you come up with some extended user, extended list of the meetings? Um no, I'm not saying you can pick a user, but you should pic